### Importação de bibliotecas

In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as pd

#caso o codigo seja executado no google colab, instalar as bibliotecas abaixo
!pip install pyspark
!pip install -q findspark
import pyarrow
import pyspark
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import csv
import matplotlib.pyplot as plt
%matplotlib inline

### Inicia uma sessão Spark

In [2]:
spark = SparkSession.builder.appName("atividade_somativa").getOrCreate()

22/06/17 16:28:28 WARN Utils: Your hostname, douglas resolves to a loopback address: 127.0.1.1; using 192.168.100.32 instead (on interface enp6s0)
22/06/17 16:28:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/17 16:28:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

### Carrega o dataset

In [4]:
df = pd.read_csv("2022-04-06_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv", sep = ";", encoding = "iso-8859-1")

### Filtra os dados que serão usados na analise

In [5]:
df_filtrado = df[["Sexo","Descrição do CID","Solicitação de Exames","Encaminhamento para Atendimento Especialista","Desencadeou Internamento","Abastecimento","Energia Elétrica","Tipo de Habitação","Destino Lixo","Fezes/Urina"]]

In [6]:
df_filtrado.info()

22/06/17 16:29:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/17 16:29:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 874894 entries, 0 to 874893
Data columns (total 10 columns):
 #   Column                                        Non-Null Count   Dtype 
---  ------                                        --------------   ----- 
 0   Sexo                                          874894 non-null  object
 1   Descrição do CID                              874613 non-null  object
 2   Solicitação de Exames                         874894 non-null  object
 3   Encaminhamento para Atendimento Especialista  874894 non-null  object
 4   Desencadeou Internamento                      874894 non-null  object
 5   Abastecimento                                 785548 non-null  object
 6   Energia Elétrica                              874894 non-null  object
 7   Tipo de Habitação                             785561 non-null  object
 8   Destino Lixo                                  785563 non-null  object
 9   Fezes/Urina                                   785561 non

### Elimina os nulos

In [7]:
df_filtrado.dropna()

22/06/17 16:29:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/17 16:29:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/17 16:29:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,Sexo,Descrição do CID,Solicitação de Exames,Encaminhamento para Atendimento Especialista,Desencadeou Internamento,Abastecimento,Energia Elétrica,Tipo de Habitação,Destino Lixo,Fezes/Urina
0,M,CUIDADO MEDICO NAO ESPECIFICADO,Sim,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,FOSSA SÉPTICA
1,F,CALCULOSE DO RIM,Sim,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
2,F,DOR EM MEMBRO,Nao,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
3,F,GRAVIDEZ (AINDA) NAO CONFIRMADA,Sim,Nao,Nao,REDE PÚBLICA,Sim,MADEIRA APARELHADA,COLETADO,SISTEMA DE ESGOTO
5,M,DOR LOMBAR BAIXA,Nao,Nao,Nao,REDE PÚBLICA,Nao,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
6,M,TRANSTORNO NAO ESPECIFICADO DA MEMBRANA DO TIM...,Sim,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
7,M,OUTR. QUEDAS MESMO NIVEL COL. OU EMP. POR OUT....,Sim,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
8,F,INFECCAO DO TRATO URINARIO DE LOCALIZACAO NAO ...,Nao,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
10,F,INSUFICIENCIA CARDIACA CONGESTIVA,Sim,Nao,Nao,OUTROS,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,OUTROS,OUTROS
11,M,INFECCAO INTESTINAL DEVIDA A VIRUS NAO ESPECIF...,Sim,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO


### filtra os cids para as doenças mais comuns

In [8]:
cids_utilizados = ["DIAGNOSTICO DE COVID-19 CONFIRMADO POR EXAMES LABORATORIAIS","HIPERTENSAO ESSENCIAL (PRIMARIA)","DIARREIA E GASTROENTERITE DE ORIGEM INFECCIOSA PRESUMIVEL","INFECCAO AGUDA DAS VIAS AEREAS SUPERIORES NAO ESPECIFICADA","DOR LOMBAR BAIXA","ANSIEDADE GENERALIZADA","CEFALEIA", "NASOFARINGITE AGUDA [RESFRIADO COMUM]","OUTRAS DORES ABDOMINAIS E AS NAO ESPECIFICADAS","CISTITE AGUDA","INFECCAO DO TRATO URINARIO DE LOCALIZACAO NAO ESPECIFICADA","GASTROENTERITE E COLITE NAO-INFECCIOSAS, NAO ESPECIFICADAS","NAUSEA E VOMITOS","AMIGDALITE AGUDA NAO ESPECIFICADA","DOR ARTICULAR","TOSSE","DOR EM MEMBRO","DOR AGUDA","DIABETES MELLITUS INSULINO-DEPENDENTE - SEM COMPLICACOES"]
df_filtrado2=df_filtrado[df_filtrado["Descrição do CID"].isin(cids_utilizados)]

In [9]:
df_filtrado2.head()

22/06/17 16:30:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/17 16:30:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/17 16:30:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,Sexo,Descrição do CID,Solicitação de Exames,Encaminhamento para Atendimento Especialista,Desencadeou Internamento,Abastecimento,Energia Elétrica,Tipo de Habitação,Destino Lixo,Fezes/Urina
2,F,DOR EM MEMBRO,Nao,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
5,M,DOR LOMBAR BAIXA,Nao,Nao,Nao,REDE PÚBLICA,Nao,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
8,F,INFECCAO DO TRATO URINARIO DE LOCALIZACAO NAO ...,Nao,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO
9,F,"GASTROENTERITE E COLITE NAO-INFECCIOSAS, NAO E...",Sim,Nao,Nao,None,Nao,None,None,None
13,F,DOR LOMBAR BAIXA,Sim,Nao,Nao,REDE PÚBLICA,Sim,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO


### Transforma a coluna "Desencadeou Internamento" em numérica para depois ser o "label" no ML

In [10]:
df_filtrado2["Desencadeou Internamento"] = df_filtrado2["Desencadeou Internamento"].replace(("Sim", "Nao"), (1,0))

### Ao final, transformaremos o dataset manipulado num arquivo .cvs. Faremos isso pq a biblioteca Mlib, utilizada na etapa seguinte, não lê o objeto pandas.

In [11]:
df_filtrado2.to_csv("dados_saude.csv", index=False, num_files = 1)

/home/douglas/anaconda3/lib/python3.9/site-packages/pyspark/pandas/generic.py:864: FutureWarning: `num_files` has been deprecated and might be removed in a future version. Use `DataFrame.spark.repartition` instead.
  warnings.warn(
